In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

### DENSENET 211

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

  Cloning https://github.com/NVIDIA/apex.git to /tmp/pip-install-gxajzqs9/apex
  Running setup.py bdist_wheel for fvcore ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/31/b5/a9/df0fd43b263cc96c1d5076ade63fdcdd30fab8699e3366a60c
  Running setup.py bdist_wheel for apex ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-b_baj90_/wheels/20/ef/9d/1967e1ee0ae20e7dc8e41ab7208017893b0a026243189508a3
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built fvcore apex termcolor
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# Need to add this to requirements.txt
!pip install tensorboard

    100% |████████████████████████████████| 3.0MB 12.5MB/s ta 0:00:01
    100% |████████████████████████████████| 19.7MB 2.1MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 34.5MB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 37.0MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 21.0MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 32.9MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 28.6MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 27.5MB/s ta 0:00:01
    100% |████████████████████████████████| 778kB 28.2MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 32.8MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 36.2MB/s ta 0:00:01
  Running setup.py bdist_wheel for grpcio ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1e/e9/13/2ce6c99171a977bad2f9a4bccfa596a6e7ea060b9fbff51bc3
  Running setup.py bdist_wheel for absl-py ... done
  Stored in d

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/densenet.yaml \
    model.densenet.depth 100 \
    model.densenet.growth_rate 12 \
    train.batch_size 64 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_2_5 \
    train.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00 \
    scheduler.epochs 400

[2020-06-11 02:51:38] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: densenet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_

In [ ]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume400_50 \
    scheduler.epochs 50


In [11]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00_resume400_50/test_results_0050_cifar10

[2020-06-11 22:25:08] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12/exp00/checkpoint_00300.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  7.93it/s]
[2020-06-11 22:25:29] __main__ INFO: Elapsed 19.79
[2020-06-11 22:25:29] __main__ INFO: Loss 0.2223 Accuracy 0.9484


In [12]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/test_results_0400_cifar10

[2020-06-11 22:26:20] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12/exp00/checkpoint_00200.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.00it/s]
[2020-06-11 22:26:40] __main__ INFO: Elapsed 19.63
[2020-06-11 22:26:40] __main__ INFO: Loss 0.2279 Accuracy 0.9456


In [13]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/exp00_resume400_50/test_results_0050_cifar101

[2020-06-11 22:27:19] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12/exp00/checkpoint_00100.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  7.99it/s]
[2020-06-11 22:27:39] __main__ INFO: Elapsed 19.65
[2020-06-11 22:27:39] __main__ INFO: Loss 0.3681 Accuracy 0.8875


In [3]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/densenet.yaml \
    model.densenet.depth 100 \
    model.densenet.growth_rate 12 \
    test.batch_size 64 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/checkpoint_00400.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/test_results_0400_cifar101

[2020-06-14 03:23:57] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12/exp00/checkpoint_00300.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.69it/s]
[2020-06-14 03:24:01] __main__ INFO: Elapsed 4.17
[2020-06-14 03:24:01] __main__ INFO: Loss 0.5098 Accuracy 0.8830


In [ ]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/test_results_0400_cifar101

In [ ]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/checkpoint_00300.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/test_results_0300_cifar101

In [4]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['densenet_BC_100_12', 'densenet_BC_100_12', 'densenet_BC_100_12', 'densenet_BC_100_12'],
           'Testset': ['cifar10', 'cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 200, 300, 300],
           'Loss': [0.3681, 0.2279, 0.2223, 0.5098],
           'Accuracy': [0.8875, 0.9456, 0.9484, 0.8830],
           'Original_Accuracy': [95.5, 95.5, 95.5, 87.6],
           'Original_CI': [(95.1, 95.9), (95.1, 95.9), (95.1, 95.9), (86.1, 89.0)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])


df.to_csv('/home/ec2-user/SageMaker/experiments/densenet_BC_100_12/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,densenet_BC_100_12,cifar10,100,0.3681,0.8875,95.5,"(95.1, 95.9)"
1,densenet_BC_100_12,cifar10,200,0.2279,0.9456,95.5,"(95.1, 95.9)"
2,densenet_BC_100_12,cifar10,300,0.2223,0.9484,95.5,"(95.1, 95.9)"
3,densenet_BC_100_12,cifar10.1,300,0.5098,0.8830,87.6,"(86.1, 89.0)"


In [ ]:
import pandas as pd
a = pd.Series(['densenet_BC_100_12_ra_2_5', 400, 'cifar10', ])
b = pd.Series(['densenet_BC_100_12_ra_2_5', 300, 'cifar10', ])
c = pd.Series(['densenet_BC_100_12_ra_2_5', 400, 'cifar10.1', ])
d = pd.Series(['densenet_BC_100_12_ra_2_5', 300, 'cifar10.1', ])


e = pd.Series(['densenet_BC_100_12_ra_2_5_refined400', 50, 'cifar10.1', ])
f = pd.Series(['densenet_BC_100_12_ra_2_5_refined400', 50, 'cifar10', ])
# g = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10', 0.4499, 0.8795])
# h = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10.1', 0.8206, 0.7710])
               
df_results = pd.concat([a,b,c,d,e,f, g, h], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 95.5 if row[2] == 'cifar10' else 87.6), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (95.1, 95.9) if row[2] == 'cifar10' else (86.1, 89.0)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/results.csv')
df_results

In [15]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5/exp00/test_results_0400/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[ 0.52143633, -2.8019223 , -0.8997567 , ..., -2.5607846 ,
        -1.9235623 , -4.5915513 ],
       [ 4.9353356 ,  7.9521174 , -6.911659  , ..., -8.307925  ,
        25.695108  ,  1.3413441 ],
       [ 0.35714385,  3.9963446 , -3.4226243 , ..., -3.5199487 ,
        16.49064   ,  1.3169571 ],
       ...,
       [-4.081325  , -3.2222219 ,  0.61520386, ..., -0.71396273,
        -2.3293254 , -4.0331235 ],
       [-0.8401726 , 12.760264  ,  3.1554806 , ..., -3.8803117 ,
        -2.1453722 , -3.1206942 ],
       [-2.0525799 ,  1.2923297 , -1.9459023 , ..., 15.644227  ,
        -2.2655675 , -0.05720376]], dtype=float32)

In [16]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-june29'
prefix = 'sagemaker/results/original-models/densenet_BC_100_12_ra_2_5'
path = '/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)